In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
import requests
from io import StringIO
import pandas as pd
root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.ingestion import cotacao_dolar_ptax
from src.features.data import criar_tabela_spark, salvar_tabela_delta_spark
from src.features.processing import renomear_colunas



In [0]:
# construção da tabela histórica de cotação do dolar ptax
API_BC_URL = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
TABELA_HISTORICA = "`api-forecasting`.bronze.dolar_ptax_historico_raw"

DATA_INICIAL = "01-01-2015"
DATA_FINAL = "11-18-2025"


df_cotacao_dolar = cotacao_dolar_ptax(
    API_BC_URL=API_BC_URL,
    DATA_INICIAL=DATA_INICIAL,
    DATA_FINAL=DATA_FINAL
)

df_cotacao_dolar.head()

In [0]:
# sanitização da tabela para persistência
cols_sanitizacao = {
'dataHoraCotacao': 'data',
'cotacaoCompra': 'cotacao_compra',
'cotacaoVenda': 'cotacao_venda'
}

df_cotacao_dolar = renomear_colunas(df= df_cotacao_dolar, 
                                    dicionario=cols_sanitizacao, df_spark=False)
                                                       
df_cotacao_dolar['data'] = pd.to_datetime(df_cotacao_dolar['data'], format='mixed')
df_cotacao_dolar.head()


In [0]:
# conversão do dataframe no formato spark
df_cotacao_dolar = criar_tabela_spark(df_cotacao_dolar)
df_cotacao_dolar.show(5)

In [0]:
# persistência da tabela na camada bronze
salvar_tabela_delta_spark(df=df_cotacao_dolar, 
                          mode='overwrite',
                          merge_schema=False,
                          partitionby=['data'], 
                          path=TABELA_HISTORICA)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.dolar_ptax_historico_raw
LIMIT 10